In [138]:
!pip install folium
import folium
import requests

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


# Loading the data

In [184]:
list_provinces = ['nl', 'pe', 'ns', 'nb', 'qc', 'on', 'mb', 'sk', 'ab', 'bc', 'yt', 'nt', 'nu']


def download_legislators(province_name = "bc", gender = "M"):
    url = "https://goverlytics.p.rapidapi.com/division-legislators/ca/" + province_name + "?limit=150"


    querystring = {"gender":"M","is_active":"true"}

    headers = {
        'x-rapidapi-host': "goverlytics.p.rapidapi.com",
        'x-rapidapi-key': "212c957c59mshe712ce12fd7352fp18875djsn03a97ed3a159"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    response_dataframe = pd.DataFrame(response.json()["data"])
    
    return response_dataframe

def download_division_legislations(province_name = "bc"):
    
    url = "https://goverlytics.p.rapidapi.com/division-legislation/ca/" + province_name

    querystring = {"include_text":"true","include_actions":"true","include_votes":"true","include_summary":"true"}

    headers = {
        'x-rapidapi-host': "goverlytics.p.rapidapi.com",
        'x-rapidapi-key': "212c957c59mshe712ce12fd7352fp18875djsn03a97ed3a159"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    response_df = pd.DataFrame(response.json()["data"])
    
    return response_df

def download_federal_legislations():
    
    url = "https://goverlytics.p.rapidapi.com/federal-legislation/ca"

    querystring = {"sponsor_gender":"M","include_votes":"true","include_actions":"true","include_text":"true","include_summary":"true"}

    headers = {
        'x-rapidapi-host': "goverlytics.p.rapidapi.com",
        'x-rapidapi-key': "212c957c59mshe712ce12fd7352fp18875djsn03a97ed3a159"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    response_df = pd.DataFrame(response.json()["data"])
    
    return response_df
   
def download_federal_legislators():
    url = "https://goverlytics.p.rapidapi.com/federal-legislators/ca"

    querystring = {"is_active":"true","gender":"M"}

    headers = {
        'x-rapidapi-host': "goverlytics.p.rapidapi.com",
        'x-rapidapi-key': "212c957c59mshe712ce12fd7352fp18875djsn03a97ed3a159"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    response_df = pd.DataFrame(response.json()["data"])
    
    return response_df
    
#Download legislators
    
dict_division_legislators_male = dict([])

dict_division_legislators_female = dict([])

for e in list_provinces:
    response = download_legislators(e, "M")
    dict_division_legislators_male[e] = response
    
for e in list_provinces:
    diccio = dict([])
    response = download_legislators(e, "F")
    dict_division_legislators_female[e] = response
    
#Download legislations

dict_division_legislations = dict([])

dict_federal_legislations = dict([])

for e in list_provinces:
    diccio = dict([])
    response = download_division_legislations(e)
    dict_division_legislations[e] = response
    
df_federal_legislations = download_federal_legislations()

df_federal_legislators = download_federal_legislators()

In [151]:
df_federal_legislations.columns

Index(['goverlytics_id', 'source_id', 'bill_name', 'session',
       'date_introduced', 'source_url', 'chamber_origin', 'committees',
       'bill_type', 'bill_title', 'country_id', 'country', 'current_status',
       'principal_sponsor_id', 'principal_sponsor', 'sponsors', 'sponsors_id',
       'cosponsors', 'cosponsors_id', 'bill_description', 'source_topic',
       'topic', 'province_territory_id', 'province_territory', 'region',
       'sponsor_affiliation', 'sponsor_gender', 'pm_name_full', 'pm_party',
       'pm_party_id', 'statute_year', 'statute_chapter', 'publications',
       'last_major_event', 'actions', 'bill_summary', 'bill_text', 'votes'],
      dtype='object')

In [152]:
len(df_federal_legislations[df_federal_legislations["last_major_event"]=="Royal Assent"])

0

In [155]:
df_federal_legislations = dict_division_legislations['bc']

#len(df_federal_legislations[df_federal_legislations["last_major_event"]=="Royal Assent"])
df_federal_legislations.columns

Index(['goverlytics_id', 'source_id', 'bill_name', 'session',
       'date_introduced', 'source_url', 'chamber_origin', 'committees',
       'bill_type', 'bill_title', 'country_id', 'country', 'current_status',
       'principal_sponsor_id', 'principal_sponsor', 'sponsors', 'sponsors_id',
       'cosponsors', 'cosponsors_id', 'bill_description', 'source_topic',
       'topic', 'province_territory_id', 'province_territory', 'actions',
       'bill_summary', 'bill_text', 'votes'],
      dtype='object')

In [48]:
df_federal_legislations.columns

Index(['goverlytics_id', 'source_id', 'bill_name', 'session',
       'date_introduced', 'source_url', 'chamber_origin', 'committees',
       'bill_type', 'bill_title', 'country_id', 'country', 'current_status',
       'principal_sponsor_id', 'principal_sponsor', 'sponsors', 'sponsors_id',
       'cosponsors', 'cosponsors_id', 'bill_description', 'source_topic',
       'topic', 'province_territory_id', 'province_territory', 'region',
       'sponsor_affiliation', 'sponsor_gender', 'pm_name_full', 'pm_party',
       'pm_party_id', 'statute_year', 'statute_chapter', 'publications',
       'last_major_event', 'actions', 'bill_summary', 'bill_text', 'votes'],
      dtype='object')

In [68]:
dict_division_legislations['bc'].columns

Index(['goverlytics_id', 'source_id', 'bill_name', 'session',
       'date_introduced', 'source_url', 'chamber_origin', 'committees',
       'bill_type', 'bill_title', 'country_id', 'country', 'current_status',
       'principal_sponsor_id', 'principal_sponsor', 'sponsors', 'sponsors_id',
       'cosponsors', 'cosponsors_id', 'bill_description', 'source_topic',
       'topic', 'province_territory_id', 'province_territory', 'actions',
       'bill_summary', 'bill_text', 'votes'],
      dtype='object')

In [71]:
dict_division_legislations['bc']['topic']

0                   housing
1                   housing
2            macroeconomics
3     government operations
4             law and crime
5         domestic commerce
6     government operations
7             law and crime
8         domestic commerce
9                     labor
10    government operations
11    government operations
12    government operations
13    government operations
14            law and crime
15                  defense
Name: topic, dtype: object

In [179]:
dict_division_legislations['ab'].topic

0                    health
1                          
2             law and crime
3                          
4                    health
5                          
6     international affairs
7                          
8                          
9     government operations
10                         
11    government operations
12                   energy
13            law and crime
14                         
15                  housing
16                   energy
17                         
18                education
19                   health
20                         
21                         
22                         
23              immigration
24                  housing
Name: topic, dtype: object

In [182]:
dict_division_legislations['on'].actions

0     [{'date': '2021-04-01', 'action_by': '', 'desc...
1     [{'date': '2021-03-31', 'action_by': '', 'desc...
2     [{'date': '2021-03-31', 'action_by': '', 'desc...
3     [{'date': '2021-03-30', 'action_by': '', 'desc...
4     [{'date': '2021-03-30', 'action_by': '', 'desc...
5     [{'date': '2021-03-23', 'action_by': '', 'desc...
6     [{'date': '2021-04-01', 'action_by': 'Standing...
7     [{'date': '2021-03-24', 'action_by': '', 'desc...
8     [{'date': '2021-04-01', 'action_by': '', 'desc...
9     [{'date': '2021-03-31', 'action_by': '', 'desc...
10    [{'date': '2021-03-23', 'action_by': '', 'desc...
11    [{'date': '2021-03-11', 'action_by': '', 'desc...
12    [{'date': '2021-03-23', 'action_by': 'Standing...
13    [{'date': '2021-03-11', 'action_by': '', 'desc...
14    [{'date': '2021-03-08', 'action_by': '', 'desc...
15    [{'date': '2021-03-08', 'action_by': '', 'desc...
16    [{'date': '2021-03-08', 'action_by': '', 'desc...
17    [{'date': '2021-04-01', 'action_by': '', '

In [177]:
dict_division_legislations['on'].date_introduced

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
Name: date_introduced, dtype: object

In [163]:
url = "https://goverlytics.p.rapidapi.com/federal-legislation/ca"

querystring = {"sponsor_gender":"M","include_votes":"true","include_actions":"true","include_text":"true","include_summary":"true"}

headers = {
    'x-rapidapi-host': "goverlytics.p.rapidapi.com",
    'x-rapidapi-key': "212c957c59mshe712ce12fd7352fp18875djsn03a97ed3a159"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

response_dataframe = pd.DataFrame(response.json()["data"])

In [167]:
import folium



canada = folium.Map(
                    location=[58.084167, -99.959314],
                    zoom_start = 3
                )

folium.Marker(location=[53.5, -124.4], popup = "<h4> <b> British Columbia</h4> </b>\\ Bills passed: ", tooltip = "BC").add_to(canada)
folium.Marker(location=[55.436645, -114.177669],popup = "<h4> <b> Alberta</h4> </b>\\ Bills passed:" , tooltip = "AB").add_to(canada)
folium.Marker(location=[55.149194, -105.988562],popup = "<h4> <b> Saskatchewan</h4> </b>\\ Bills passed:" , tooltip = "SK").add_to(canada)
folium.Marker(location=[55.222793, -97.115573], popup = "<h4> <b> Manitoba</h4> </b>\\ Bills passed:" , tooltip= "MB").add_to(canada)
folium.Marker(location=[51.008623, -86.090106], popup = "<h4> <b> Ontario</h4> </b>\\ Bills passed:" , tooltip= "ON").add_to(canada)
folium.Marker(location=[52.069473, -71.427672], popup = "<h4> <b> Quebec</h4> </b>\\ Bills passed:" , tooltip= "QB").add_to(canada)
folium.Marker(location=[53.400751, -60.730773], popup = "<h4> <b> Newfoundland and Labrador</h4> </b>\\ Bills passed:" , tooltip= "NL").add_to(canada)
folium.Marker(location=[64.245580, -136.177762],popup = "<h4> <b> Yukon</h4> </b>\\ Bills passed:" , tooltip = "YT").add_to(canada)
folium.Marker(location=[46.389358, -63.302173], popup = "<h4> <b> Prince Edward</h4> </b>\\ Bills passed:" , tooltip= "PE").add_to(canada)
folium.Marker(location=[45.273052, -63.035154], popup = "<h4> <b> Nova Scotia</h4> </b>\\ Bills passed:" , tooltip= "NS").add_to(canada)
folium.Marker(location=[46.648582, -66.396753], popup = "<h4> <b> New Brunswick</h4> </b>\\ Bills passed:" , tooltip= "NB").add_to(canada)
folium.Marker(location=[66.372019, -94.719900], popup = "<h4> <b> Nunavut</h4> </b>\\ Bills passed:" , tooltip= "NU").add_to(canada)
folium.Marker(location=[64.042750, -116.565396],popup = "<h4> <b> Northern Territories</h4> </b>\\ Bills passed:" , tooltip = "NT").add_to(canada)


canada